In [2]:
#datos
import requests
import pandas as pd

#para manipular fechas
import arrow
from datetime import datetime, timedelta

#gráficos
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from dash import Dash, dcc, html

# Precios de mercados, a ultimo momento, con cotizaciónes y últimos movimientos

In [3]:
respuesta = requests.get("https://ftx.com/api/markets")

respuesta = respuesta.json()["result"] #respuesta al estilo una lista de diccionarios

mercado = pd.DataFrame(respuesta)

# Precios históricos de la moneda seleccionada

In [4]:
#establecemos una fecha a 4 meses
fechanalisis=datetime.now()-timedelta(120)

#Seleccionamos nuestras variables con las que vamos a trabajar
seleccion="BTC/USD"
intervalo=86400
fechaini=arrow.get(fechanalisis).timestamp()

#preparamos la dirección para que se pueda usar con cualquier seleccion
url = f"https://ftx.com/api/markets/{seleccion}/candles"

#generamos los argumentos que vamos a usar
argumentos= {"resolution":intervalo,
       "start_time":fechaini}

#hacemos la consulta
respuesta = requests.get(url, params=argumentos)
#Nos quedamos con la parte result del json que obtenemos de respuesta
respuesta = respuesta.json()['result']

#Generamos nuestro df
monedas = pd.DataFrame(respuesta)

#convertimos a formato datetime la fecha que nos brinda la consulta
monedas.startTime = pd.to_datetime(monedas.startTime)

#Generamos las medias móviles a 99, 25 y 7
monedas["MA99"]=monedas.close.rolling(window=99).mean()
monedas["MA25"]=monedas.close.rolling(window=25).mean()
monedas["MA7"]=monedas.close.rolling(window=7).mean()

#configuramos nuestro diccionario de colores
colores = {
    'velasdown': '#EB6D88',
    'velasup': '#23A4F2',
    'fondograf': '#252138',
    'fondotodo':'#2A263D',
    'fondodiv':'3C3956'
}


#Creamos un grafico de velas con los datos recolectados
figura = make_subplots(rows=2, cols=1, shared_xaxes=True, 
                        vertical_spacing=0.1, 
                        subplot_titles=(f"Evolución de {seleccion} en dólares", 
                                        'Volumen de transacciones'),
                        row_width=[0.2, 0.7],
                      )

#creamos las Velas
figura.add_trace(go.Candlestick(x=monedas.startTime,
                                open=monedas.open,
                                high=monedas.high,
                                low=monedas.low,
                                close=monedas.close,
                                showlegend=False,
                                increasing_line_color= colores['velasup'], 
                                decreasing_line_color= colores['velasdown']
                                ), row=1, col=1)

def añadirMA(seriedf,color):
  figura.add_trace(go.Scatter(x=monedas.startTime,
                              y=seriedf,
                              line=dict(color=color), #se pueden añadir más detalles a la linea
                              name=seriedf.name
                              ), row=1, col=1
                  )

añadirMA(monedas["MA99"],"#9A94B8")
añadirMA(monedas["MA25"],"#F1CB81")
añadirMA(monedas["MA7"],"#ED6F85")


#creamos un color para el volumen de las velas
monedas['color']=[colores['velasup'] if (x<y) else colores['velasdown'] for x,y in zip(monedas['open'],monedas['close'])]

#Creamos los Volumenes
figura.add_trace(go.Bar(x=monedas.startTime, 
                  y=monedas.volume, 
                  marker_color=monedas['color'],
                  showlegend=False), row=2, col=1)

#sacamos la barra de navegación que se encuentra debajo
figura.update_layout(xaxis_rangeslider_visible = False)
                    #paper_bgcolor=colors['fondotodo'], #cambia el color del cuadro contenedor
                    #plot_bgcolor=colors['fondograf'] #cambia el color de lo mas peque
          

figura.show()

In [5]:
"""app = dash.Dash()
app.layout = html.Div([
    dcc.Graph(figure=figura) #simplemente pasamos la figura creada
])

app.run_server(debug=True, use_reloader=False) #reloader false en jupyter"""

'app = dash.Dash()\napp.layout = html.Div([\n    dcc.Graph(figure=figura) #simplemente pasamos la figura creada\n])\n\napp.run_server(debug=True, use_reloader=False) #reloader false en jupyter'

Vamos a seleccionar como trabajo únicamente los mercados al contado, para simplificar el análisis y concentrarnos en la visualización.

In [6]:
#Nos quedamos con el mercadoa a contado y el cambio con theter a cambio estable
mercado = mercado.loc[(mercado["type"]=="spot") & mercado.name.str.contains('USDT')]

In [7]:
mercado

,name,enabled,postOnly,priceIncrement,sizeIncrement,minProvideSize,last,bid,ask,price,...,highLeverageFeeExempt,largeOrderThreshold,change1h,change24h,changeBod,quoteVolume24h,volumeUsd24h,priceHigh24h,priceLow24h,tokenizedEquity
11,AAVE/USDT,True,False,1.000000e-02,0.01,0.01,7.676000e+01,7.655000e+01,7.667000e+01,7.667000e+01,...,True,672.0,-0.008663,0.038607,-0.006479,97459.451400,97449.705455,7.775000e+01,7.300000e+01,NaN
24,AKRO/USDT,True,False,5.000000e-06,1.00,1.00,3.960000e-03,3.960000e-03,3.970000e-03,3.960000e-03,...,True,196.0,-0.005025,0.046235,0.001264,1347.014605,1346.879904,4.075000e-03,3.775000e-03,NaN
32,ALGO/USDT,True,False,1.000000e-04,1.00,1.00,3.603000e-01,3.594000e-01,3.598000e-01,3.598000e-01,...,True,672.0,0.001113,0.071152,0.026241,80820.363500,80812.281464,3.637000e-01,3.322000e-01,NaN
48,AMPL/USDT,True,False,1.000000e-04,1.00,1.00,1.143000e+00,1.143600e+00,1.151400e+00,1.143600e+00,...,True,196.0,-0.000699,0.070887,-0.006774,38813.667500,38809.786133,1.163300e+00,1.066200e+00,NaN
70,ATOM/USDT,True,False,2.500000e-03,0.10,0.10,1.285500e+01,1.285500e+01,1.286750e+01,1.285500e+01,...,True,1400.0,-0.013431,-0.013431,-0.008293,550800.661500,550745.581434,1.332000e+01,1.262500e+01,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
913,VETBULL/USDT,True,False,1.000000e-08,1000.00,1000.00,8.849000e-05,8.586000e-05,8.717000e-05,8.717000e-05,...,True,196.0,-0.014917,0.140372,-0.017913,82.930890,82.922597,8.895000e-05,7.630000e-05,NaN
922,XRPBEAR/USDT,True,False,1.000000e-08,10000000.00,10000000.00,9.000000e-08,5.000000e-08,1.700000e-07,9.000000e-08,...,True,196.0,0.000000,0.000000,0.000000,0.000000,0.000000,9.000000e-08,9.000000e-08,NaN
924,XRPBULL/USDT,True,False,1.000000e-08,10000.00,10000.00,2.957000e-05,2.934000e-05,2.964000e-05,2.957000e-05,...,True,196.0,-0.038374,0.168775,-0.061568,7597.462660,7596.702914,3.209000e-05,2.479000e-05,NaN
928,XTZBEAR/USDT,True,False,1.000000e-08,1000000.00,1000000.00,1.300000e-07,1.400000e-07,1.600000e-07,1.400000e-07,...,True,196.0,0.076923,-0.066667,0.000000,0.000000,0.000000,1.500000e-07,1.300000e-07,NaN


In [8]:
mercado[mercado.name.str.contains('ETH')]

,name,enabled,postOnly,priceIncrement,sizeIncrement,minProvideSize,last,bid,ask,price,...,highLeverageFeeExempt,largeOrderThreshold,change1h,change24h,changeBod,quoteVolume24h,volumeUsd24h,priceHigh24h,priceLow24h,tokenizedEquity
278,ETH/USDT,True,False,1.000000e-01,0.001,0.001,1.337400e+03,1.337400e+03,1.337500e+03,1.337400e+03,...,True,1680.0,-0.007937,0.052822,-0.003131,7.204080e+07,7.203360e+07,1.355300e+03,1.254000e+03,NaN
817,ETHBEAR/USDT,True,False,1.000000e-08,1000000.000,1000000.000,5.000000e-08,7.000000e-08,6.000000e-08,6.000000e-08,...,True,196.0,0.200000,0.200000,0.200000,0.000000e+00,0.000000e+00,6.000000e-08,5.000000e-08,NaN
819,ETHBULL/USDT,True,False,1.000000e-03,0.010,0.010,4.576000e+00,4.564000e+00,4.581000e+00,4.576000e+00,...,True,196.0,-0.022849,0.171531,-0.007375,4.206761e+05,4.206340e+05,4.744000e+00,3.800000e+00,NaN


In [9]:
monedas

,startTime,time,open,high,low,close,volume,MA99,MA25,MA7,color
0,2022-06-01 00:00:00+00:00,1.654042e+12,31779.0,31958.0,29300.0,29779.0,1.039113e+09,NaN,NaN,NaN,#EB6D88
1,2022-06-02 00:00:00+00:00,1.654128e+12,29779.0,30677.0,29554.0,30432.0,7.082379e+08,NaN,NaN,NaN,#23A4F2
2,2022-06-03 00:00:00+00:00,1.654214e+12,30432.0,30678.0,29267.0,29671.0,5.890949e+08,NaN,NaN,NaN,#EB6D88
3,2022-06-04 00:00:00+00:00,1.654301e+12,29671.0,29955.0,29453.0,29842.0,2.223259e+08,NaN,NaN,NaN,#23A4F2
4,2022-06-05 00:00:00+00:00,1.654387e+12,29842.0,30179.0,29503.0,29897.0,2.549769e+08,NaN,NaN,NaN,#23A4F2
...,...,...,...,...,...,...,...,...,...,...,...
116,2022-09-25 00:00:00+00:00,1.664064e+12,18922.0,19188.0,18632.0,18809.0,2.248359e+08,21262.050505,19883.52,19041.142857,#EB6D88
117,2022-09-26 00:00:00+00:00,1.664150e+12,18809.0,19318.0,18687.0,19228.0,7.497010e+08,21248.787879,19847.36,18997.285714,#23A4F2
118,2022-09-27 00:00:00+00:00,1.664237e+12,19228.0,20387.0,18820.0,19077.0,1.102962e+09,21233.848485,19812.40,19026.000000,#EB6D88
119,2022-09-28 00:00:00+00:00,1.664323e+12,19078.0,19769.0,18464.0,19414.0,9.184522e+08,21220.818182,19795.80,19162.714286,#23A4F2


# Calculadora

In [10]:
calculadora=mercado[['name','price']]

def convert(desde, hasta):

    #conseguimos el precio de la moneda ingresada
    entrada=float(calculadora[calculadora.name==desde]['price'].values)
    destino=float(calculadora[calculadora.name==hasta]['price'].values)
    salida=entrada/destino

    print(f'Un {desde} vale {salida:.3f} {hasta}')

convert('BTC/USDT','ETH/USDT')

Un BTC/USDT vale 14.517 ETH/USDT


In [14]:
float(mercado[mercado.name=='BTC/USDT']['price'].values)

19415.0